In [1]:
%load_ext autoreload

In [2]:
%autoreload

import h2o
import pandas as pd
import utilities as utils

from h2o.automl import H2OAutoML

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
data_list = utils.read_data()
Data = utils.DataProcessor(data_list)
df = Data.create_train_data()
df.sort_index(inplace=True)

,direct_rad:W,elevation:m,sun_elevation:d,sun_azimuth:d,air_density_2m:kgm3,sfc_pressure:hPa,wind_speed_u_10m:ms,cloud_base_agl:m,ceiling_height_agl:m,visibility:m,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,wind_speed_10m:ms,effective_cloud_cover:p,wind_speed_w_1000hPa:ms,pv_measurement
date_forecast,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0,7.0,-49.139999,12.985000,1.239,986.599976,1.1,1226.400024,1226.400024,19275.900391,89.199997,2.2,2.5,69.599998,-0.0,0.0
2019-01-01 01:00:00,0.0,7.0,-46.497002,33.275002,1.241,985.599976,1.3,1157.599976,1157.599976,12777.299805,88.900002,3.1,3.4,98.099998,-0.0,0.0
2019-01-01 02:00:00,0.0,7.0,-41.963001,51.573002,1.238,985.099976,2.0,1087.599976,1087.599976,12776.700195,87.599998,3.3,3.8,96.300003,-0.0,0.0
2019-01-01 03:00:00,0.0,7.0,-36.167000,67.794998,1.240,985.299988,2.4,1016.400024,1016.400024,8399.900391,86.800003,3.6,4.3,97.199997,-0.0,0.0
2019-01-01 04:00:00,0.0,7.0,-29.681999,82.383003,1.239,985.700012,5.0,1029.000000,1029.000000,4102.799805,91.000000,3.2,5.9,97.500000,-0.0,0.0


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /home/stinky/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpwib8sxj_
  JVM stdout: /tmp/tmpwib8sxj_/h2o_stinky_started_from_python.out
  JVM stderr: /tmp/tmpwib8sxj_/h2o_stinky_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_stinky_uhe8qp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [10]:
X_train = df[utils.get_features(4)]

train = h2o.H2OFrame(X_train)

y = 'pv_measurement'

X = train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
['direct_rad:W', 'elevation:m', 'sun_elevation:d', 'sun_azimuth:d', 'air_density_2m:kgm3', 'sfc_pressure:hPa', 'wind_speed_u_10m:ms', 'cloud_base_agl:m', 'ceiling_height_agl:m', 'visibility:m', 'relative_humidity_1000hPa:p', 'wind_speed_v_10m:ms', 'wind_speed_10m:ms', 'effective_cloud_cover:p', 'wind_speed_w_1000hPa:ms']


In [24]:
aml = H2OAutoML(max_models=30, seed=0)
aml.train(x=X, y=y, training_frame=train)

AutoML progress: |██ (cancelled)

13:25:15.606: Empty leaderboard.
AutoML was not able to build any model within a max runtime constraint of 0 seconds, you may want to increase this value before retrying.



H2OJobCancelled: Job<$03017f00000132d4ffffffff$_afa6a24539383d087c986c538f164f1e> was cancelled by the user.

In [12]:
test_pd = Data.create_test_data()
# Remove features that are not in the training set
test_pd = test_pd[X]

test = h2o.H2OFrame(test_pd)

prediction = aml.predict(test)

,direct_rad:W,elevation:m,sun_elevation:d,sun_azimuth:d,air_density_2m:kgm3,sfc_pressure:hPa,wind_speed_u_10m:ms,cloud_base_agl:m,ceiling_height_agl:m,visibility:m,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,wind_speed_10m:ms,effective_cloud_cover:p,wind_speed_w_1000hPa:ms
date_forecast,,,,,,,,,,,,,,,
2023-05-01 00:00:00,0.0,6.0,-11.185,10.520,1.286,1013.299988,2.2,1041.199951,912.700012,30210.699219,80.300003,3.4,4.0,80.699997,-0.0
2023-05-01 00:00:00,0.0,7.0,-11.184,10.521,1.283,1013.200012,2.2,1039.900024,912.299988,31329.500000,80.300003,3.4,4.0,80.699997,-0.0
2023-05-01 00:00:00,0.0,24.0,-11.223,10.519,1.286,1010.799988,1.5,1178.000000,NaN,31411.000000,81.099998,3.1,3.5,73.699997,-0.0
2023-05-01 00:15:00,0.0,6.0,-10.825,14.203,1.287,1013.200012,2.1,1054.800049,912.700012,30003.599609,80.300003,3.4,4.0,77.000000,-0.0
2023-05-01 00:15:00,0.0,7.0,-10.824,14.204,1.283,1013.000000,2.1,1053.099976,912.299988,31181.599609,80.300003,3.4,4.0,77.099998,-0.0


In [ ]:
preds_df = prediction.as_data_frame()
preds_df[preds_df < 0] = 0

results = pd.concat([test_pd.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'h2o')